In [5]:
import tweepy
import pandas as pd
from twitter_api_xanda import TWITTER_API_KEY, TWITTER_API_SECRET, TWITTER_API_BEARER

In [52]:
company_handle = "yumbrands"
FIELDS = ["created_at", "text", "public_metrics", "context_annotations", "entities", "referenced_tweets"]

In [53]:
client = tweepy.Client(bearer_token=TWITTER_API_BEARER)

user = client.get_user(username=company_handle)
tweets = client.get_users_tweets(id=user.data.id, tweet_fields=FIELDS)

In [94]:
def parse_entities(tweet):
    info_tuples = set()
    if "annotations" not in tweet.entities:
        return info_tuples
    
    for obj in tweet.entities['annotations']:
        annotation_type = obj['type']
        annotation_text = obj['normalized_text']
        
        info_tuples.add((annotation_type, annotation_text))

    return info_tuples

def parse_context_annotations(tweet):
    info_tuples = set()
    for obj in tweet.context_annotations:
        domain_id = obj['domain']['id']
        domain_name = obj['domain']['name']
        entity_name = obj['entity']['name']
        
        info_tuples.add((domain_id, domain_name, entity_name))
        
    return info_tuples

In [91]:
tweet = tweets.data[0]
metrics = tweet.public_metrics

tweet_id = tweet.id
created_at = str(tweet.created_at)
text = tweet.text
like_count = metrics["like_count"]
reply_count = metrics["reply_count"]
retweet_count = metrics["retweet_count"]
referenced_tweets = tweet.referenced_tweets
context_annotations = parse_context_annotations(tweet)
entities = parse_entities(tweet)

{('131', 'Unified Twitter Taxonomy', 'Yum! Brands'), ('30', 'Entities [Entity Service]', 'Fast food Restaurants'), ('131', 'Unified Twitter Taxonomy', 'S&P 500'), ('47', 'Brand', 'Pizza Hut'), ('47', 'Brand', 'USAID'), ('47', 'Brand', 'Yum! Brands'), ('152', 'Food', 'Fast food Restaurants'), ('66', 'Interests and Hobbies Category', 'Cuisines'), ('30', 'Entities [Entity Service]', 'QSR - Dining'), ('45', 'Brand Vertical', 'Government/Education'), ('30', 'Entities [Entity Service]', 'Food'), ('46', 'Business Taxonomy', 'Food & Beverage Business'), ('30', 'Entities [Entity Service]', 'Non-profit - Government/Education'), ('131', 'Unified Twitter Taxonomy', 'Pizza Hut'), ('45', 'Brand Vertical', 'Dining')}


In [92]:
COLUMNS = ["tweet_id", "created_at", "text", "like_count", "reply_count", "retweet_count", "referenced_tweets", "context_annotations", "entities"]
def get_tweets_for_user(username: str):
    user = client.get_user(username=username)
    tweets = client.get_users_tweets(id=user.data.id, tweet_fields=FIELDS)
    
    rows = []
    
    for tweet in tweets.data:
        tweet_id = tweet.id
        created_at = str(tweet.created_at)
        text = tweet.text
        like_count = metrics["like_count"]
        reply_count = metrics["reply_count"]
        retweet_count = metrics["retweet_count"]
        referenced_tweets = tweet.referenced_tweets
        context_annotations = parse_context_annotations(tweet)
        entities = parse_entities(tweet)
        
        rows.append([tweet_id, created_at, text, like_count, reply_count, retweet_count, referenced_tweets, context_annotations, entities])
        
    df = pd.DataFrame(rows, columns=COLUMNS)
    
    return df

In [95]:
yum_tweets_df = get_tweets_for_user("yumbrands")

yum_tweets_df.to_csv("yum_tweets.csv")